**Resources Visited**

https://machinelearningmastery.com/develop-word-embeddings-python-gensim/

https://radimrehurek.com/gensim/models/word2vec.html

https://radimrehurek.com/gensim/models/doc2vec.html

In [1]:
# !pip install gensim
# !pip install --upgrade pandas
# !pip install unidecode


In [1]:
import pickle
import pandas as pd
import numpy as np
pd.set_option('max_rows',200)

pd.__version__ # 1.3 was necessary for the pickle read to run

'1.3.5'

In [8]:
# Reading the Pickle file
filename = 'Organizations.xlsx'

df = pd.read_excel('Organizations.xlsx')

In [14]:

df['Interest'] =  df['DOMAINE DINTERET'].copy()


In [19]:
df.Interest = df.Interest.dropna()

In [21]:
# Extracting the raw corpus
corpus = df.Interest.to_list()
corpus = [str(i).lower() for i in corpus]   
df_corpus = pd.DataFrame(corpus, columns = ['corpus1'])

In [22]:
corpus[:5]

['éducation, environnement, santé et  bien-être seulement.',
 'cancer, hôpitaux, maladies du coeur et  santé.',
 'éducation, santé, bien-être, vétérans.',
 'jeunesse (local seulement).',
 'centraide, développement économique  et social, droits de la personne, éducation, enfants et jeunes, environnement, santé et sécurité.']

## 1. NLP Preprocessing

In [23]:
# nltk.download('wordnet')
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re

stopwords = stopwords.words('french') + stopwords.words('english') + ['/' ,'-', 'pour', 'ou', 'du', 'la', 'ou', 'de',
         'avec', 'par', 'depuis','bien','a', 'd\'un', '']

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords] #french stopwords
    return " ".join(filtered_words)

In [24]:
corpus2 = pd.Series(corpus).map(lambda s:preprocess(s)) 

df_corpus['corpus2'] = corpus2

In [25]:
corpus2

0            éducation environnement santé être seulement
1                    cancer hôpitaux maladies coeur santé
2                           éducation santé être vétérans
3                                jeunesse local seulement
4       centraide développement économique social droi...
                              ...                        
1337    appui org communautaires éducation peuples aut...
1338    engagement auprès communautés cris québec envi...
1339    causes charitables général région bois francs ...
1340    appuie plusieurs activités financement locales...
1341                            organismes communautaires
Length: 1342, dtype: object

In [70]:
# !pip install sentencepiece

In [107]:
# !pip install -U sentence-transformers


In [112]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')
# 1st: find best lang model


Downloading: 100%|██████████| 114/114 [00:00<00:00, 30.8kB/s]
Downloading: 100%|██████████| 1.58M/1.58M [00:00<00:00, 17.0MB/s]
Downloading: 100%|██████████| 2.38k/2.38k [00:00<00:00, 1.15MB/s]
Downloading: 100%|██████████| 122/122 [00:00<00:00, 62.8kB/s]
Downloading: 100%|██████████| 539M/539M [00:23<00:00, 23.2MB/s] 
Downloading: 100%|██████████| 53.0/53.0 [00:00<00:00, 17.8kB/s]
Downloading: 100%|██████████| 112/112 [00:00<00:00, 20.4kB/s]
Downloading: 100%|██████████| 1.96M/1.96M [00:00<00:00, 12.7MB/s]
Downloading: 100%|██████████| 452/452 [00:00<00:00, 75.3kB/s]
Downloading: 100%|██████████| 996k/996k [00:00<00:00, 7.37MB/s]
Downloading: 100%|██████████| 341/341 [00:00<00:00, 85.2kB/s]


In [116]:
sentences = corpus3.tolist()

In [129]:
embeddings = [model.encode(i) for i in sentences]

In [200]:
from scipy import spatial

def get_similarity(x,y):
    result = 1 - spatial.distance.cosine(x, y)

    return result

In [202]:

sim_scores = []


for s1, e1 in zip(sentences, embeddings):
    for s2, e2 in zip(sentences, embeddings):

        if s1 == s2:
            continue

        sim_score = get_similarity(e1, e2)

        row = [s1, s2, sim_score]

        sim_scores.append(row)

        



In [204]:
df_similarity = pd.DataFrame(data=sim_scores, columns = ['Sentence1', 'Sentence2', 'Similarity'])

In [209]:
df_similarity.sort_values(['Sentence1', 'Similarity'], ascending=False).drop_duplicates()

,Sentence1,Sentence2,Similarity
1179205,être sociétal environnement jeux génie québec fondation cancer sein fondation sourdine maison pignon bleu,fondation centre cancérologie charles bruneau fondation mélio jour terre québec,0.609712
1179148,être sociétal environnement jeux génie québec fondation cancer sein fondation sourdine maison pignon bleu,fondation hôpital général lakeshore fondation institut gériatrie montréal société alzheimer maison victor gadbois société cancer etc,0.586582
1179063,être sociétal environnement jeux génie québec fondation cancer sein fondation sourdine maison pignon bleu,appuie fondation canadienne cancer sein commandite course vie cibc environnement sécurité,0.583719
1178628,être sociétal environnement jeux génie québec fondation cancer sein fondation sourdine maison pignon bleu,centraide fondation cancer sein québec opération père noël société canadienne cancer fondation centre jeunesse québec,0.578907
1178745,être sociétal environnement jeux génie québec fondation cancer sein fondation sourdine maison pignon bleu,rêves enfants société canadienne cancer relais vie jeux québec,0.576309
...,...,...,...
98849,accès emploi activités culturelles centraide éducation entraide bécancour environnement loisirs santé sécurité économique services sociaux,évènements spéciaux,-0.022450
97779,accès emploi activités culturelles centraide éducation entraide bécancour environnement loisirs santé sécurité économique services sociaux,causes femmes,-0.031423
98507,accès emploi activités culturelles centraide éducation entraide bécancour environnement loisirs santé sécurité économique services sociaux,aînés,-0.039549
97805,accès emploi activités culturelles centraide éducation entraide bécancour environnement loisirs santé sécurité économique services sociaux,nan,-0.041951


In [194]:
sentences[0]

'éducation environnement santé être seulement'

In [195]:
sentences[1]

'cancer hôpitaux maladies coeur santé'

In [130]:
corpus3['emb'] = embeddings

In [166]:
# !pip install umap-learn

In [132]:
import umap
umap_embeddings = umap.UMAP(n_neighbors=15, 
                            n_components=5, 
                            metric='cosine').fit_transform(embeddings)

In [180]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4, random_state=0).fit(embeddings) #7

In [181]:
df['interest_domain_cluster'] = kmeans.labels_

In [182]:
df['interest_domain_cluster'].value_counts(normalize=True)*100

3    40.760060
2    21.162444
0    20.789866
1    17.287630
Name: interest_domain_cluster, dtype: float64

In [188]:
df.groupby('interest_domain_cluster')['IS_PAST_DONOR'].sum()/df.IS_PAST_DONOR.sum()

interest_domain_cluster
0    0.163636
1    0.127273
2    0.163636
3    0.545455
Name: IS_PAST_DONOR, dtype: float64

In [189]:
pd.set_option('display.max_colwidth', None)



In [185]:
df.loc[ df['interest_domain_cluster'] == 0, 'DOMAINE DINTERET'].drop_duplicates()

0                  Éducation, environnement, santé et  bien-être seulement.
1                            Cancer, hôpitaux, maladies du coeur et  santé.
2                                    Éducation, Santé, Bien-être, Vétérans.
3                                               Jeunesse (local seulement).
5       Maladies tropicales, négligées et  dangereuses, éducation, animaux.
                                       ...                                 
1303                             Environnement, développement et recherche.
1324                          Privilégie Centraide Laurentides et la santé.
1328                               Privilégie le domaine de  l'agriculture.
1330                                              Environnement, Centraide.
1334                    Privilégie la santé et organismes locaux seulement.
Name: DOMAINE DINTERET, Length: 212, dtype: object

In [151]:
df_pd = pd.read_pickle('dataset.pickle')

In [155]:
pd_index = df_pd.ID.drop_duplicates().tolist()

In [156]:
pd_index

[1312,
 1278,
 1139,
 1334,
 271,
 67,
 846,
 489,
 1142,
 1182,
 1244,
 742,
 366,
 289,
 337,
 237,
 236,
 178,
 704,
 114,
 1091,
 373,
 138,
 433,
 58,
 511,
 261,
 1043,
 1108,
 400,
 1167,
 212,
 851,
 823,
 759,
 483,
 588,
 639,
 811,
 53,
 726,
 137,
 723,
 555,
 1040,
 87,
 235,
 362,
 730,
 971,
 677,
 259,
 121,
 1286,
 110]

In [158]:
df['IS_PAST_DONOR'] = 0
df.loc[df.ID.isin(pd_index), 'IS_PAST_DONOR'] = 1

In [162]:
numerator = df.groupby('interest_domain_cluster')['IS_PAST_DONOR'].sum()

interest_domain_cluster
0     5
1    36
2     4
3     1
4     0
5     6
6     3
Name: IS_PAST_DONOR, dtype: int64

In [164]:
df.groupby('interest_domain_cluster')['IS_PAST_DONOR'].count()

interest_domain_cluster
0    185
1    545
2     84
3    131
4     54
5    292
6     51
Name: IS_PAST_DONOR, dtype: int64

In [165]:
df.to_clipboard(index=None)

In [210]:
df

,ID,2E CONTACT POUR,ADDRESS,AVIS,CONTACT,CONTRIBUTION,COURRIEL,DDD,DOMAINE DINTERET,FAF,...,NBRE DE SUCC,NOMBRE DEMPLOYES,NOTE,POSTE,PRINCIP FILIALES,SECTEUR INDUSTRIEL,SITE WEB,Interest,interest_domain_cluster,IS_PAST_DONOR
0,1,NaN,"7290, rue Frederick Banting Saint-Laurent QC H4T1Z2","Dons autres qu'en argent. Dons en « temps » (bénévolat). Faire les demandes par : courrier, courriel.","Madame Leanne Hayward, Superviseur aux communications",Dons & commandite,lhayward@mmm.com,En tout temps,"Éducation, environnement, santé et bien-être seulement.",31 décembre,...,NaN,100,Toute demande de dons ou de commandites doit être envoyée à Mme Hayward par la poste ou par courriel à : lhayward@mmm.com.,2390,NaN,"Équipement et services industriels; Fabrication de peintures, de revêtements et de rubans adhésifs, abrasifs, produits de soins de santé et télécommunications.",https://www.3mcanada.ca/,"Éducation, environnement, santé et bien-être seulement.",0,0
1,2,"DonsMadame Marie-Josée Dery, Coordonnatrice Capital Humain","305, 12e Avenue Richelieu QC J3L3T2",Faire les demandes par : courrier.,"Monsieur Claude Cardin, Président",Dons,mjdery@prevost-architectural.com; info@prevost-architectural.com,En tout temps,"Cancer, hôpitaux, maladies du coeur et santé.",31 décembre,...,NaN,200,NaN,NaN,NaN,"Matériaux de construction, manufacturiers; Fabrication de produits d'architecture, d'éléments de charpentes métalliques et traitement anodique.",www.adprevost.ca,"Cancer, hôpitaux, maladies du coeur et santé.",0,0
2,3,NaN,"3750, chemin de l'aéroport Trois-Rivières QC G9B2N8",Faire les demandes par : courriel.,"Madame Elizabeth Vautour, Adjointe exécutive au vice-président des opérations",Dons,elizabeth.vautour@aarcorp.com; comptabilite@aarcorp.com,En tout temps,"Éducation, Santé, Bien-être, Vétérans.",31 décembre,...,NaN,340,"Envoyer votre demande de dons par courriel à Mme Vautour, Adjoint au V.P. des opérations à : elizabeth.vautour@aarcorp.com.",3201,NaN,"Offre des services d'entretien, de réparation et de révision (MRO) d'aéronefs.",https://www.aarcorp.com,"Éducation, Santé, Bien-être, Vétérans.",0,0
3,4,NaN,800 boul. Hymes Saint-Laurent QC H4S0B5,"Faire les demandes par : courrier, fax.","Madame Carley Tietolman, Responsable des dons et commandites",Dons & commandite,NaN,En tout temps,Jeunesse (local seulement).,NaN,...,NaN,160,Toute demande de dons ou de commandites doit être envoyée par courrier ou par télécopieur et adressez votre demande au responsable des don ou des commandites.,NaN,NaN,"Équipement et services industriels; Fabriquer des poulies, bagues, poulies et roues dentées synchrones, poulies étagées et accouplements élastomères de la plus haute qualité possible dans le marché.",http://new.abb.com/ca,Jeunesse (local seulement).,0,0
4,5,NaN,800 boul. Hymes Saint-Laurent QC H4S0B5,Faire les demandes par : courrier.,"Monsieur Guido Mussehl, Contrôleur Finances",Dons & commandite,"guido.mussehl@ca.abb.comMadame Nathalie Arsenault, Assist. au Président ; contact.center@ca.abb.com; nathalie.m.arsenault@ca.abb.com",En tout temps,"Centraide, développement économique et social, droits de la personne, éducation, enfants et jeunes, environnement, santé et sécurité.",31 décembre,...,NaN,1 400,Par courrier seulement.,NaN,NaN,"Commerce de gros et détail; Grossistes-distributeurs de produits pétroliers, équipement, technologie, transport d'énergie, technologie de l'automatisation, gaz, pétrochimie.",www.ABB.com/ca,"Centraide, développement économique et social, droits de la personne, éducation, enfants et jeunes, environnement, santé et sécurité.",3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1337,1338,NaN,"3400, boul. de Maisonneuve O., bur. 900 Montréal QC H3Z3G1","Dons en « temps » (bénévolat). Faire les demandes par : courrier, fax.","Madame Cynthia Laberge, Adjointe Marketing",Dons,cynthia.laberge@xerox.com,En tout temps,"Appui aux org. communautaires, éducation, peuples autochtones, femmes, personne